In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI


load_dotenv()

llm = ChatOpenAI(model="qwen-max")

# 单轮提示

In [2]:
single_turn_prompt = "三原色是哪三种颜色？"

print(llm.invoke(single_turn_prompt).content)

三原色是指能够通过不同比例的混合生成其他颜色的基础颜色。根据应用领域的不同，三原色可以分为加法三原色和减法三原色两大类：

1. **加法三原色**：主要应用于发光体的颜色混合，如电视、电脑屏幕等显示设备。加法三原色指的是红（Red）、绿（Green）、蓝（Blue），简称RGB。这三种颜色以不同强度组合可以产生各种色彩。

2. **减法三原色**：主要用于颜料或染料的颜色混合，比如印刷业中使用的油墨。减法三原色指的是青（Cyan）、品红（Magenta）以及黄（Yellow），有时也会加上黑色（Black）来增强对比度和深度，这就是CMYK色彩模型的基础。

在日常交流中如果不特别指明领域，默认情况下提到的“三原色”通常是指加法三原色即红、绿、蓝(RGB)。


In [ ]:
from langchain_core.prompts import PromptTemplate


structured_prompt = PromptTemplate(
    input_variables=["topic"], template="简要说明{topic}并列出其三个主要组成部分。"
)

chain = structured_prompt | llm
print(chain.invoke({"topic": "颜色的原理"}).content)

颜色的原理主要基于光与物质相互作用以及人眼对这种相互作用结果的感受。当光源发出的光线照射到物体上时，物体会吸收某些波长（颜色）的光，并反射或透射其他波长的光。我们看到的颜色实际上是物体反射或透射出来的光的颜色。例如，如果一个物体看起来是红色的，那是因为它反射了红光而吸收了其他颜色的光。

在色彩理论中，颜色有三个主要组成部分，通常被称为色彩三要素或颜色模型中的三个维度：

1. **色相（Hue）**：指的是纯色的基本属性，也就是我们通常所说的“颜色”，如红色、蓝色等。它是根据光谱上的位置来定义的，反映了颜色的基本类型。
2. **饱和度（Saturation）**：也称为纯度或彩度，描述的是颜色的纯净程度或鲜艳程度。饱和度越高，颜色看起来越纯正、越鲜艳；饱和度低则意味着颜色更加接近灰色调。
3. **亮度（Brightness）或明度（Value）**：指颜色的明亮程度，即从黑色到白色之间的变化。对于给定的色相和饱和度，通过调整亮度可以得到不同深浅的效果。

这三者共同决定了一个颜色的具体表现形式。不同的组合会产生无数种颜色变化，从而构成了丰富多彩的世界。此外，在不同的应用领域（如印刷、电子显示等），可能还会使用特定的颜色模型来表示这三个组成部分，比如RGB（红绿蓝）、CMYK（青品黄黑）等。


# 多轮提示（会话）

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph


# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
from langchain_core.messages import HumanMessage


config = {"configurable": {"thread_id": "abc123"}}

multi_turn_prompts = [
    "你好，我正在学习太空知识。你能告诉我有关行星的知识吗？",
    "我们的太阳系中最大的行星是什么？",
    "与地球相比的话，它有多大？",
]

for prompt in multi_turn_prompts:
    input_messages = [HumanMessage(prompt)]
    output = app.invoke({"messages": input_messages}, config)
    # output contains all messages in state
    output["messages"][-1].pretty_print()

================================== Ai Message ==================================

当然可以！行星是围绕恒星（如我们的太阳）运行的天体，具有足够的质量以自身的重力克服刚体力，从而达到流体静力平衡（近似球形），并且已经清除了其轨道附近的其他物体。根据国际天文联合会2006年的定义，一颗行星必须满足以下三个条件：

1. 绕着太阳公转。
2. 有足够大的质量来克服固体应力，使其形状接近于流体静力平衡状态下的形状（通常是球形或略扁的球形）。
3. 已经清空了它所在轨道上的区域，即在其轨道上没有其他大量小天体存在。

在我们太阳系中，目前被正式认定为行星的一共有八颗，按照离太阳的距离从近到远依次为：水星、金星、地球、火星、木星、土星、天王星和海王星。这八大行星又可以根据它们的特点分为两大类：

- **岩石行星**：包括水星、金星、地球和火星。这些行星主要由岩石或金属构成，体积相对较小，密度较高。
- **气态巨行星**：包括木星、土星、天王星和海王星。这类行星拥有巨大的气体包层，主要是氢和氦组成的大气层，以及可能存在的液体内部结构。其中，天王星和海王星因为含有较多的冰物质（如水、氨和甲烷），有时也被称作“冰巨星”。

此外，还有许多其他有趣的天体绕太阳运行，比如矮行星（例如冥王星）、小行星带中的小行星、柯伊伯带及奥尔特云中的彗星等，它们虽然不符合行星的标准定义，但也是太阳系内不可或缺的一部分。

如果你对某个特定的行星或者关于行星探索的历史感兴趣的话，请告诉我，我很乐意提供更多相关信息！
================================== Ai Message ==================================

我们的太阳系中最大的行星是木星。木星是一颗气态巨行星，其体积和质量都非常庞大。具体来说：

- **体积**：木星的体积大约是地球的1,321倍。
- **质量**：它的质量约为地球的318倍。
- **直径**：木星的赤道直径约为139,820公里（约86,881英里），是太阳系中直径最大的行星。

木星主要由氢和氦组成，拥有强大的磁场和至少79颗已知的卫星。其中最著名的四颗伽利略卫星分别是：伽利莫德、欧罗巴、甘尼米德和卡利斯托。此外，木星还以其

# 单轮 VS 多轮

In [8]:
prompts = ["法国首都在哪？", "它有多少人口？", "这个城市最著名的地标是什么？"]


print("单轮回复:")
for prompt in prompts:
    print(f"Q: {prompt}")
    print(f"A: {llm.invoke(prompt).content}\n")

# Multi-turn prompts
print("多轮回复:")
config = {"configurable": {"thread_id": "abc456"}}
for prompt in prompts:
    print(f"Q: {prompt}")
    output = app.invoke({"messages": [HumanMessage(prompt)]}, config)
    print(f"A: {output['messages'][-1].content}\n")

单轮回复:
Q: 法国首都在哪？
A: 法国的首都是巴黎(Paris)。

Q: 它有多少人口？
A: 您的问题中没有提到具体的“它”是指哪个地方或国家，所以我无法直接给出答案。如果您能提供更详细的信息，比如具体的城市名、省份名或国家名，我就能帮助您找到相关的人口数据了。

Q: 这个城市最著名的地标是什么？
A: 您的问题中没有提到具体是哪个城市，不同的城市有着不同的著名地标。例如：

- 如果您指的是北京，那么最著名的地标可能是天安门广场和故宫。
- 对于上海来说，东方明珠塔是一个非常知名的地标。
- 在巴黎，埃菲尔铁塔无疑是该市最标志性的建筑之一。
- 纽约则以其自由女神像闻名于世。

请告诉我您想了解哪个城市的地标，我将为您提供更详细的信息。

多轮回复:
Q: 法国首都在哪？
A: 法国的首都是巴黎（Paris）。

Q: 它有多少人口？
A: 截至2021年的数据，巴黎市内的人口大约为216万左右。如果考虑整个巴黎大区（Île-de-France），包括周边的郊区，人口则超过1200万。请注意，这些数字会随着时间而有所变化。如果您需要最新的人口统计数据，建议查阅最新的官方发布资料。

Q: 这个城市最著名的地标是什么？
A: 巴黎最著名的地标无疑是埃菲尔铁塔（Tour Eiffel）。这座铁塔建于1889年，是为了纪念法国大革命一百周年和展示工业技术成就而建造的。它高约324米（包括天线），是巴黎乃至整个法国的象征之一。

除了埃菲尔铁塔之外，巴黎还有许多其他著名的地标，例如：
- 卢浮宫（Louvre）：世界上最大、最著名的艺术博物馆之一。
- 巴黎圣母院（Notre-Dame de Paris）：一座哥特式建筑风格的大教堂，虽然在2019年遭受火灾，但仍然是巴黎的重要标志。
- 凯旋门（Arc de Triomphe）：位于香榭丽舍大道的一端，为纪念拿破仑战争胜利而建。
- 蒙马特高地（Montmartre）：以其艺术家社区、圣心大教堂（Basilique du Sacré-Cœur）和美丽的风景而闻名。
- 塞纳河（Seine River）：流经巴黎市中心，沿岸有许多历史悠久的建筑和桥梁。

这些地标不仅代表了巴黎的历史和文化，也是游客们必访的地方。

